In [1]:
import os
import pickle
from transformers import GPTNeoXForCausalLM, GPTNeoXConfig, GPTNeoXTokenizerFast
from generate import load_model
from datasets import load_from_disk
import collections
import torch

/local/home/antonis/.conda/envs/incidental/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-12 14:23:54.897097: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
ds_path = "/share/edc/home/antonis/datasets/huggingface/flan_v1_task_ds"
ds  = load_from_disk(ds_path)

In [ ]:
device = "cuda:0"
model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/gpt-neox-20b").half().to(device)

In [ ]:
tokenizer = GPTNeoXTokenizerFast.from_pretrained("EleutherAI/gpt-neox-20b")

In [ ]:
"""
dict levels:

task -> prompt / target / generated -> n_generations -> generation

"""

gen_dict = collections.defaultdict(
           lambda: collections.defaultdict(lambda: collections.defaultdict(dict))
)

save_path = "./incidental-supervision/results/generations/gpt-neo-x"

for task in ds.keys():
    # sample 10 rows
    ds_task = ds[task].select(range(10))
    for idx, row in enumerate(ds_task):
        prompt = row["inputs"]
        targets = row["targets"]
        # concatenate
        input_ = prompt + " " + targets
        # split into two
        input_1, input_2 = input_[:len(input_)//2], input_[len(input_)//2:]
        gen_dict[task]["prompt"] = input_1
        gen_dict[task]["continuation"] = input_2
        # tokenize
        input_1_ = tokenizer(input_1, return_tensors="pt").input_ids.to(device)
        input_2_ = tokenizer(input_2, return_tensors="pt").input_ids.to(device)
        
        n_samples = 10
        # generate 10 samples
        print(f"--- True continuation: {input_2} ---")
        for i in range(n_samples):
            gen = model.generate(input_1_, max_new_tokens=int(len(input_2)*1.2), 
                                 do_sample=True, top_p=0.95, 
                                 num_return_sequences=1)
            gen_dec = tokenizer.decode(gen[0])
            gen_dict[task][idx]["generated"][i] = gen_dec
            print(f"{i}: {gen_dec}")
            # clear server
            torch.cuda.empty_cache()
            # save dict as pkl
            # with open(os.path.join(save_path, "gpt-neo-x.pkl"), "wb") as f:
                # pickle.dump(gen_dict, f)